# Build a Game Playing Agent - Knights Isolation
## James McGuigan
- Source: https://github.com/JamesMcGuigan/udacity-artificial-intelligence/blob/master/Projects/3_Adversarial%20Search/

# Unit Tests

In [1]:
from agents.AlphaBetaPlayer import AlphaBetaAreaPlayer, AlphaBetaPlayer, MinimaxPlayer
from agents.DistancePlayer import DistancePlayer, GreedyDistancePlayer
from agents.MCTS import MCTSMaximum, MCTSMaximumHeuristic, MCTSRandom, MCTSRandomHeuristic
from agents.UCT import UCTPlayer
from sample_players import GreedyPlayer, RandomPlayer
from run_backpropagation import run_backpropagation
from isolation import Agent, logger
from run_match_sync import play_sync


%load_ext autoreload
%autoreload 2

In [2]:
! python3 -m unittest -v

test_get_action_midgame (tests.test_my_custom_player.CustomPlayerGetActionTest)
get_action() calls self.queue.put() before timeout in a game in progress ... ok
test_get_action_player1 (tests.test_my_custom_player.CustomPlayerGetActionTest)
get_action() calls self.queue.put() before timeout on an empty board ... ok
test_get_action_player2 (tests.test_my_custom_player.CustomPlayerGetActionTest)
get_action() calls self.queue.put() before timeout as player 2 ... ok
test_get_action_terminal (tests.test_my_custom_player.CustomPlayerGetActionTest)
get_action() calls self.queue.put() before timeout when the game is over ... ok
test_custom_player (tests.test_my_custom_player.CustomPlayerPlayTest)
CustomPlayer successfully completes a game against itself ... ok

----------------------------------------------------------------------
Ran 5 tests in 16.348s

OK


# Infrastructure

- [run_match_sync.py](run_match_sync.py)
- [run_backpropagation.py](run_backpropagation.py)
- [agents/DataSavePlayer.py](agents/DataSavePlayer.py)

I've rewritten `run_match.py` with a synchronous implementation in `run_match_sync.py` and `run_backpropagation.py`.
using signals rather than `multithreading.Pool()`, which allows for better profiling and a 2x performance speedup.

I have improved the CLI flags and logging ouput, with an extra `--verbose` flag which can
be used to ASCII print out the board state after each turn.

DataSavePlayer now handles loading and atexit autosaving of cls.data, whilst also gzipping contents


# Advanced Heuristic

- [agents/AlphaBetaPlayer.py](agents/AlphaBetaPlayer.py)

> What features of the game does your heuristic incorporate, and why do you think those features matter in evaluating states during search?

The main analogy is with the game of Go. The goal is to surround your opponent and capture a larger territory.

Recursively computing liberties several moves ahead shows the area of the board that the opponent
could potentially escape to.

At `depth=1` this `heuristic_area()` is equivalent to `#my_moves - #opponent_moves`

Early in the game the opponent effectively has access to the entire board, making this heuristic ineffective,
hence `max_area` is used in addition to depth to shortcircuit the computational cost of
expanding breadth-first search to all possible future moves on a mostly empty board when neither side is trapped.

This heuristic is endgame focused. It solves the simplified subproblem of local search without an adversary,
and provides an upper-bound estimate for the difference in maximum number of total moves each player has remaining.
It reaches maximum value for moves that trap a player within a self-contained section whilst leaving
the other a means of escape. The player in the smaller territory will run out of moves first.

The other major impact on the performance of this agent is the addition of alphabeta pruning
with the aggressive use of caching. This avoids the computation expense of recomputing previously explored subtrees, and by caching on `@classmethod`
parts of the cache can even be reused between runs. The net effect of this is to increase the
maximum depth of iterative deepening before the timeout to beyond what `MinimaxPlayer(depth=3)` can compute.

Improvements over previous submission:
- Iterative Deepening checks terminates early if action score is infinite
- Caching has been redone: now keyed on (player_id,state) and ignores depth/alpha/beta
- Alphabeta caching only stores infinite values, which is effectively an endgame table
- Caching is persisted to disk ./data/*.zi  p.pickle via DataSavePlayer base class
- Persisted caching means Alphabeta can pretrained with a higher timeout before the match

Performance improvements mean the algorithm now dominates the course Minimax implementation
- First percentage is the total winrate, second percentage is the rolling average winrate

In [13]:
!rm -f ./data/A*.pickle

Greedy vs Minimax depends on who gets the first turn

In [14]:
! python3 ./run_backpropagation.py  -a GREEDY -o MINIMAX --progress -r 70

+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+- match_id:   70 |   9s |  50% ->  49% | Greedy vs Minimax


AlphaBeta gets 90%+ winrate vs both Greedy and Minimax

In [15]:
! python3 ./run_backpropagation.py  -a ALPHABETA -o GREEDY --progress -r 70

+-+++++++++++++-+++-+++++++++++++-+++++-++++++++++++++++++++++++++++++ match_id:   70 | 246s |  93% ->  96% | AlphaBeta vs Greedy
wrote:  ./data/AlphaBetaPlayer.zip.pickle        |  0.7MB in  1.2s | entries: 45237


In [16]:
! python3 ./run_backpropagation.py  -a ALPHABETA -o MINIMAX --progress -r 70

loaded: ./data/AlphaBetaPlayer.zip.pickle        |  0.7MB in  0.3s | entries: 45237
+++++-+-+-+-+-+-+++-++++++-++++++++++++++-+++++++++--+++++++++++++++++ match_id:   70 | 260s |  84% ->  90% | AlphaBeta vs Minimax
wrote:  ./data/AlphaBetaPlayer.zip.pickle        |  1.4MB in  1.9s | entries: 91139


AlphaBetaAreaPlayer gets near 100% winrate vs Greedy and Minimax, plus 70% vs AlphaBetaPlayer

In [17]:
! python3 ./run_backpropagation.py  -a AREA -o GREEDY --progress -r 70

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++-++++++++++++ match_id:   70 | 247s |  99% ->  98% | AlphaBeta Area vs Greedy
wrote:  ./data/AlphaBetaAreaPlayer.zip.pickle    |  4.9MB in  7.0s | entries: 306855


In [18]:
! python3 ./run_backpropagation.py  -a AREA -o MINIMAX --progress -r 70

loaded: ./data/AlphaBetaAreaPlayer.zip.pickle    |  4.9MB in  1.8s | entries: 306855
+-++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ match_id:   70 | 243s |  99% -> 100% | AlphaBeta Area vs Minimax
wrote:  ./data/AlphaBetaAreaPlayer.zip.pickle    |  8.2MB in 11.8s | entries: 516356


In [19]:
! python3 ./run_backpropagation.py  -a AREA -o ALPHABETA --progress -r 70

loaded: ./data/AlphaBetaPlayer.zip.pickle        |  1.4MB in  0.5s | entries: 91139
loaded: ./data/AlphaBetaAreaPlayer.zip.pickle    |  8.2MB in  3.3s | entries: 516356
+++--+++++-+++-+++++++++--++++++++--+--+--++++++++++-++--+-++-++++-++- match_id:   70 | 632s |  73% ->  70% | AlphaBeta Area vs AlphaBeta
wrote:  ./data/AlphaBetaPlayer.zip.pickle        |  3.1MB in  4.6s | entries: 202838
wrote:  ./data/AlphaBetaAreaPlayer.zip.pickle    | 15.3MB in 23.0s | entries: 969287


# Depth Analysis

- MiniMax course default is depth 3
- AlphaBeta
    - has trouble getting past depth 1 on the first turn
    - can get to depth 5 on the second turn
    - remains at depth 4-6 for the early game
    - grows to depth 6-9 during the mid-game
    - expands out to max depth of 14 before finding a -inf lose condition
- AlphaBetaArea
    - is usually depth-2 compared to AlphaBeta
    - the area heuristic effectively adds an extra 4 layers of hidden depth
    - so for the same CPU cost, AlphaBetaArea has a depth 2 advantage
    - finds the inf win condition before AlphaBeta and at a lower depth

In [28]:
AlphaBetaPlayer.verbose_depth     = True
AlphaBetaAreaPlayer.verbose_depth = True

play_sync( ( Agent(AlphaBetaPlayer,'AlphaBeta'), Agent(AlphaBetaAreaPlayer,'AlphaBetaArea') ) )

AlphaBetaPlayer.save()
AlphaBetaAreaPlayer.save()


AlphaBetaV           | depth: 1 
AlphaBetaAreaV       | depth: 1 
AlphaBetaV           | depth: 1 2 3 4 5 
AlphaBetaAreaV       | depth: 1 2 3 
AlphaBetaV           | depth: 1 2 3 4 5 
AlphaBetaAreaV       | depth: 1 2 3 4 
AlphaBetaV           | depth: 1 2 3 4 5 
AlphaBetaAreaV       | depth: 1 2 3 
AlphaBetaV           | depth: 1 2 3 4 
AlphaBetaAreaV       | depth: 1 2 3 
AlphaBetaV           | depth: 1 2 3 4 
AlphaBetaAreaV       | depth: 1 2 3 
AlphaBetaV           | depth: 1 2 3 4 5 
AlphaBetaAreaV       | depth: 1 2 3 
AlphaBetaV           | depth: 1 2 3 4 5 
AlphaBetaAreaV       | depth: 1 2 3 
AlphaBetaV           | depth: 1 2 3 4 5 
AlphaBetaAreaV       | depth: 1 2 3 
AlphaBetaV           | depth: 1 2 3 4 5 
AlphaBetaAreaV       | depth: 1 2 3 
AlphaBetaV           | depth: 1 2 3 4 
AlphaBetaAreaV       | depth: 1 2 3 
AlphaBetaV           | depth: 1 2 3 4 5 
AlphaBetaAreaV       | depth: 1 2 
AlphaBetaV           | depth: 1 2 3 4 5 6 
AlphaBetaAreaV       | depth: 1 2 3 4 

We can pretrain AlphaBeta with a 10 minute timeout to fully explore the opening position and optimal game strategy, and build up a killer startgame cache, which should allow AlphaBeta to now beat AlphaBetaArea

In [ ]:
AlphaBetaPlayer.verbose_depth = True
run_backpropagation({
    'agent':     'ALPHABETA',
    'opponent':  'ALPHABETA',
    'rounds':     1,
    'timeout':    10*60,     # seconds
    'time_limit': 10*60*100, # milliseconds
})
AlphaBetaPlayer.save()


AlphaBetaPlayer      | depth: 1 2 3 4 5 6 7 8 9 

In [ ]:
play_sync( ( Agent(AlphaBetaPlayer,'AlphaBeta'), Agent(AlphaBetaAreaPlayer,'AlphaBetaArea') ) )

AlphaBetaPlayer.save()
AlphaBetaAreaPlayer.save()
AlphaBetaPlayer.verbose_depth     = False
AlphaBetaAreaPlayer.verbose_depth = False

# Monty Carlo Tree Search

- [agents/MCTS.py](agents/MCTS.py)
- [agents/UCT.py](agents/UCT.py)

I have implemented variations on Monty Carlo Tree Search.

## MCTSMaximum

is designed to be reinforcement learning agents,
to be trained by running repeatedly before the match. Initially they will make random moves.
After each match, the `.backpropergate()` function is called, which will build a record of
the win/loss ratio for each seen board position, and also compute the BestChild score
`(w/n + c√(ln N/n))` for each seen node. The agent is runtime fast in the sense that it only
needs to read from it's own cache to compute the max score of available actions, which assumes
the agent has seen this exact board position before during training,

## MCTSRandom

Is a variation inspired by the Ant Colony Optimization Algorithm.
It removes the exploration term, and instead of selecting the action with the maximum score,
it uses the difference in score as random weighting factor to stochastically select the next action.

## MCTSRandomHeuristic + MCTSMaximumHeuristic

These are variants that add the liberties heuristic to the BestChild score


## Results

- MCTSRandom wins against MCTSMaximum by 68%

In [52]:
! rm -f ./data/MCTS*.pickle
! python3 ./run_backpropagation.py  -a MCR  -o MCM  -r 100 -l 100

 match_id:  100 |   1s |  74% ->  74% | MCTS Random vs MCTS Maximum
wrote:  ./data/MCTSMaximum.zip.pickle            |  0.1MB in  0.1s | entries: 4281
wrote:  ./data/MCTSRandom.zip.pickle             |  0.1MB in  0.1s | entries: 4281


MCTSRandom can be shown to be trainable against a Greedy opponent

In [53]:
! python3 ./run_backpropagation.py  -a MCR -o GREEDY -r 1000  -l 100

loaded: ./data/MCTSRandom.zip.pickle             |  0.1MB in  0.0s | entries: 4281
 match_id:  100 |   1s |  35% ->  44% | MCTS Random vs Greedy
 match_id:  200 |   2s |  36% ->  40% | MCTS Random vs Greedy
 match_id:  300 |   4s |  38% ->  40% | MCTS Random vs Greedy
 match_id:  400 |   5s |  40% ->  43% | MCTS Random vs Greedy
 match_id:  500 |   6s |  43% ->  47% | MCTS Random vs Greedy
 match_id:  600 |   8s |  44% ->  48% | MCTS Random vs Greedy
 match_id:  700 |   9s |  46% ->  50% | MCTS Random vs Greedy
 match_id:  800 |  10s |  45% ->  48% | MCTS Random vs Greedy
 match_id:  900 |  12s |  45% ->  48% | MCTS Random vs Greedy
 match_id: 1000 |  13s |  46% ->  50% | MCTS Random vs Greedy
wrote:  ./data/MCTSRandom.zip.pickle             |  0.6MB in  0.8s | entries: 29813


MCTSRandom can almost match Minimax when trained for a sufficently long time, but has a hard time against AlphaBeta even with a reduced timer

In [54]:
! python3 ./run_backpropagation.py  -a MCR -o MINIMAX -r 1000

loaded: ./data/MCTSRandom.zip.pickle             |  0.6MB in  0.1s | entries: 29813
 match_id:  100 |  14s |  31% ->  28% | MCTS Random vs Minimax
 match_id:  200 |  27s |  33% ->  33% | MCTS Random vs Minimax
 match_id:  300 |  40s |  34% ->  35% | MCTS Random vs Minimax
 match_id:  400 |  52s |  36% ->  38% | MCTS Random vs Minimax
 match_id:  500 |  65s |  38% ->  40% | MCTS Random vs Minimax
 match_id:  600 |  78s |  40% ->  43% | MCTS Random vs Minimax
 match_id:  700 |  91s |  40% ->  43% | MCTS Random vs Minimax
 match_id:  800 | 104s |  41% ->  43% | MCTS Random vs Minimax
 match_id:  900 | 117s |  42% ->  46% | MCTS Random vs Minimax
 match_id: 1000 | 130s |  42% ->  45% | MCTS Random vs Minimax
wrote:  ./data/MCTSRandom.zip.pickle             |  1.0MB in  1.3s | entries: 51073


In [57]:
! python3 ./run_backpropagation.py  -a MCR -o ALPHABETA -r 250 -l 50 --time_limit 50

loaded: ./data/MCTSRandom.zip.pickle             |  1.0MB in  0.3s | entries: 51073
 match_id:  100 | 124s |   1% ->   1% | MCTS Random vs AlphaBeta
wrote:  ./data/AlphaBetaPlayer.zip.pickle        |  0.6MB in  0.6s | entries: 37887
wrote:  ./data/MCTSRandom.zip.pickle             |  1.1MB in  1.4s | entries: 55693


What if we pretrain all the Monty Carlo agents against each other?

In [ ]:
from run_backpropagation import TEST_AGENTS, run_backpropagation
import gc
import time
for agent in TEST_AGENTS.keys():
    for opponent in TEST_AGENTS.keys():
        if agent.startswith('MC') and opponent.startswith('MC'):
            time.sleep(0.1)
            TEST_AGENTS[agent   ].agent_class.verbose = False
            TEST_AGENTS[opponent].agent_class.verbose = False
            run_backpropagation({
                "agent":      agent,
                "opponent":   opponent,
                "rounds":     1000,
                "logging":    1000,
                "progress":   False,
                "time_limit": 150
            })

for agent in TEST_AGENTS.keys():
    for opponent in TEST_AGENTS.keys():
        if agent.startswith('MC') and opponent.startswith('MC'):
            TEST_AGENTS[opponent].agent_class.save()

Then attempt a rematch against AlphaBeta

In [ ]:
! python3 ./run_backpropagation.py  -a MCR -o ALPHABETA -r 250 -l 50 --time_limit 50

## UCTPlayer

While MCTSMaximum and MCTSRandom are fast agents, UCTPlayer will use its 150ms of time to simulate
MCTSMaximum from the current board position before returning an answer.

In [ ]:
! python3 ./run_backpropagation.py  -a UCT -o GREEDY -r 250 -l 50

In [ ]:
! python3 ./run_backpropagation.py  -a UCT -o MINIMAX -r 250 -l 50

In [ ]:
! python3 ./run_backpropagation.py  -a UCT -o ALPHABETA -r 250 -l 50 --time_limit 50

# Opening Book

## Monty Carlo Method

We can derive an opening book by looking at the cached scores in MCTSRandom
- score priorities exploration of 100% winrate nodes that have only been explored once
    - sorting by score actually suggested the best countermove was the corner square
    - sort by wins * wins/count to find the best move

- Monty Carlo suggests the best opening strategy is:
    - defend the corner on the 3,3 point
    - attack the knights blind spot on the 4,3 point
        - knight requires three turns to move one space sideways
    - the first player is attempting to move into position to directly attack their opponent
    - the second player is attempting to remain in the blind spot adjcent to the first player

In [72]:
from agents.MCTS import MCTSRandom
from isolation.isolation import Isolation, DebugState
MCTSRandom.load()

def best_response(parent_state=Isolation(), verbose=True):
    actions      = parent_state.actions()
    child_states = [ parent_state.result(action) for action in actions ]
    child_states = [ child for child in child_states if child in MCTSRandom.data ]
    records      = [ MCTSRandom.data[child] for child in child_states  ]
    best_state, best_record = max(zip(child_states, records),
                                  key=lambda key_value: key_value[1].wins * key_value[1].wins/key_value[1].count)
    best_move    = DebugState.ind2xy(best_state.locs[parent_state.player()])
    if verbose:
        print('Best move '+str(best_state.ply_count)+' is:')
        print(best_move, best_record)
        print( DebugState.from_state(best_state).bitboard_string )
        print( DebugState.from_state(best_state) )
    return best_move, best_record, best_state

best_state = Isolation()
for i in range(10):
    try:
        best_move, best_record, best_state = best_response(best_state)
    except: pass


Best move 1 is:
(2, 2) MCTSRecord(wins=697, count=1277, score=0.5454545454545454)
1111111111100111111111110011111111111001111111111100111111111110011111111111001111111101100111111111110011111111111

+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   | 1 |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |  

# AlphaBetaArea Method

Alteratively we could simply watch a precached AlphaBeta play against itself when given 1 minute per move

In [ ]:
AlphaBetaPlayer.verbose_depth = True
play_sync(
    Agent(AlphaBetaPlayer,'AlphaBeta 1'),
    Agent(AlphaBetaPlayer,'AlphaBeta 2'),
    { "verbose": True, "time_limit": 10*60*1000 }
)
AlphaBetaPlayer.save()

# League Tables

Lets run every agent against every other agent and compare results

In [92]:
gc.collect()
for agent in TEST_AGENTS.keys():
    for opponent in TEST_AGENTS.keys():
        time.sleep(0.1)
        TEST_AGENTS[agent   ].agent_class.verbose = False
        TEST_AGENTS[opponent].agent_class.verbose = False
        is_slow = any(
            name in TEST_AGENTS[agent].name + TEST_AGENTS[opponent].name
            for name in ['Alpha', 'Area', 'UCT']
        )
        run_backpropagation({
            "agent":      agent,
            "opponent":   opponent,
            "rounds":     10  if is_slow else 100,
            "progress":   False,
            "time_limit": 150    # increase to prevent TimeoutErrors
        })
    print()

for agent in TEST_AGENTS.keys():
    try: TEST_AGENTS[agent].agent_class.save()
    except: pass

 match_id:   10 |   0s |  40% ->  42% | Random vs Random 2
 match_id:   10 |   0s |  40% ->  32% | Random vs Greedy
 match_id:   10 |   0s |  50% ->  49% | Random vs Distance
 match_id:   10 |   0s |   0% ->   0% | Random vs Greedy Distance
 match_id:   10 |   3s |  10% ->  17% | Random vs Minimax
 match_id:   10 |  36s |  10% ->  19% | Random vs AlphaBeta
 match_id:   10 |  32s |   0% ->   0% | Random vs AlphaBeta Area
 match_id:   10 |   0s |  60% ->  54% | Random vs MCTS Maximum
 match_id:   10 |   0s |  40% ->  42% | Random vs MCTS Random
 match_id:   10 |   0s |  90% ->  87% | Random vs MCTS Maximum Heuristic
 match_id:   10 |   0s |  60% ->  62% | Random vs MCTS Random Heuristic
 match_id:   10 |  37s |  30% ->  41% | Random vs UCT

 match_id:   10 |   0s |  70% ->  75% | Greedy vs Random
 match_id:   10 |   0s |  50% ->  45% | Greedy vs Greedy 2
 match_id:   10 |   0s |  50% ->  55% | Greedy vs Distance
 match_id:   10 |   0s |  50% ->  45% | Greedy vs Greedy Distance
 match_id:

 match_id:   10 |   1s |  70% ->  69% | MCTS Random Heuristic vs Distance
 match_id:   10 |   0s |  20% ->  22% | MCTS Random Heuristic vs Greedy Distance
 match_id:   10 |   3s |  20% ->  10% | MCTS Random Heuristic vs Minimax
 match_id:   10 |  37s |  20% ->  18% | MCTS Random Heuristic vs AlphaBeta
 match_id:   10 |  43s |   0% ->   0% | MCTS Random Heuristic vs AlphaBeta Area
 match_id:   10 |   2s |  50% ->  49% | MCTS Random Heuristic vs MCTS Maximum
 match_id:   10 |   5s |  40% ->  28% | MCTS Random Heuristic vs MCTS Random
 match_id:   10 |   2s |  70% ->  79% | MCTS Random Heuristic vs MCTS Maximum Heuristic
 match_id:   10 |   2s |  50% ->  51% | MCTS Random Heuristic vs MCTS Random Heuristic 2
 match_id:   10 |  34s |  50% ->  55% | MCTS Random Heuristic vs UCT

 match_id:   10 |  39s |  20% ->   6% | UCT vs Random
 match_id:   10 |  37s |  30% ->  47% | UCT vs Greedy
 match_id:   10 |  44s |  70% ->  75% | UCT vs Distance
 match_id:   10 |  37s |  20% ->  26% | UCT vs Gree

AttributeError: type object 'RandomPlayer' has no attribute 'save'